In [ ]:
from recbole.data import create_dataset, data_preparation
from recbole.config import Config

In [ ]:
model='NPE'
dataset='ml-100k'

config_dict = {
    'eval_args': {
        "order": "TO",
        "split": {"RS": [0.8, 0.1, 0.1]},
        "group_by": None
    },
    'train_neg_sample_args': None
}

config = Config(
    model=model,
    dataset=dataset,
    config_dict=config_dict
)

In [ ]:
dataset = create_dataset(config)
# train_data, valid_data, test_data = data_preparation(config, dataset)
# model_type = config["MODEL_TYPE"]
# built_datasets = dataset.build()
# train_dataset, valid_dataset, test_dataset = built_datasets